# Teil 12: Trainieren eins verschlüsselten NN auf verschlüsselten Daten

In diesem Notebook werden alle bisher erarbeiteten Techniken zusammengeführt um ein verschlüsseltes Neuronales Netzwerk auf verschlüsselten Daten zu trainieren und mit ihm Vorhersagen zu tätigen. 

Insbesondere wird dabei die "Autograd-Engine" vorgestellt, welche verschlüsselte Berechnungen durchführen kann.

Autoren:
- Andrew Trask - Twitter: [@iamtrask](https://twitter.com/iamtrask)
- Jason Paumier - Github: [@Jasopaum](https://github.com/Jasopaum)
- Théo Ryffel - Twitter: [@theoryffel](https://twitter.com/theoryffel)

Übersetzer:
- Jan Moritz Behnken - Github: [@JMBehnken](https://github.com/JMBehnken)

# Schritt 1: Erstellen der Helfer und des Datensatzes

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import syft as sy

In [ ]:
# Set everything up
hook = sy.TorchHook(torch) 

alice = sy.VirtualWorker(id="alice", hook=hook)
bob = sy.VirtualWorker(id="bob", hook=hook)
james = sy.VirtualWorker(id="james", hook=hook)

In [ ]:
# A Toy Dataset
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]])
target = torch.tensor([[0],[0],[1],[1.]])

# A Toy Model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 2)
        self.fc2 = nn.Linear(2, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x
model = Net()

# Schritt 2: Verschlüsseln des Models und der Daten

Die Verschlüsselung geschieht hier in zwei Schritten. Weil "Secure Multi-Party Computation" nur mit Integern funktionert, müssen alle Dezimalzahlen (wie z. B. die Gewichte und Aktivierungsfunktionen) mit festgelegter Genauigkeit umgewandelt werden. Dafür wird die Funktion `.fix_precision()` verwendet.

Daraufhin kann `.share()` wie auch in den vorherigen Tutorials verwendet werden, um die Tensoren auf Alice und Bob verschlüsselt aufzuteilen. Zu beachten ist, dass `requires_grad` auf `True` gesetzt werden muss. Dadurch wird eine spezielle Autograd Methode zu den verschlüsselten Daten hinzugefügt. Tatsächlich kann das normale Autograd von PyTorch wegen den Integern der "Secure Multi-Party Computation" nicht verwendet werden. Aus diesem Grund muss ein spezialisierter AutogradTensor-Knoten eingefügt werden, welcher für das Backpropagieren die Gradienten berechnet. Jedes Element kann begutachtet werden, um den enthaltenen AutogradTensor zu betrachten. 

In [ ]:
# We encode everything
data = data.fix_precision().share(bob, alice, crypto_provider=james, requires_grad=True)
target = target.fix_precision().share(bob, alice, crypto_provider=james, requires_grad=True)
model = model.fix_precision().share(bob, alice, crypto_provider=james, requires_grad=True)

In [ ]:
print(data)

# Schritt 3: Training

Das Netzwerk kann nun mit simpler Tensor-Logik trainiert werden.

In [ ]:
opt = optim.SGD(params=model.parameters(),lr=0.1).fix_precision()

for iter in range(20):
    # 1) erase previous gradients (if they exist)
    opt.zero_grad()

    # 2) make a prediction
    pred = model(data)

    # 3) calculate how much we missed
    loss = ((pred - target)**2).sum()

    # 4) figure out which weights caused us to miss
    loss.backward()

    # 5) change those weights
    opt.step()

    # 6) print our progress
    print(loss.get().float_precision())

Tatsächlich konnte der Loss verringert werden!

## Effekt von festgelegter Genauigkeit

Es bleibt zu klären wie sehr das Verschlüsseln das Netzwerk-Training beeinfllusst. Da aus theoretischer Sicht die Berechnungen identisch sind, sind auch die finalen Ergebnisse sehr nah am nicht-verschlüsselten Training. Dies kann leicht überprüft werden mit einem unverschlüsselten und deterministischen Training dieses Models. 

`__init__`:
```
with torch.no_grad():
    self.fc1.weight.set_(torch.tensor([[ 0.0738, -0.2109],[-0.1579,  0.3174]], requires_grad=True))
    self.fc1.bias.set_(torch.tensor([0.,0.1], requires_grad=True))
    self.fc2.weight.set_(torch.tensor([[-0.5368,  0.7050]], requires_grad=True))
    self.fc2.bias.set_(torch.tensor([-0.0343], requires_grad=True))
```

Die kleinen Unterschiede kommen aus den Rundungen der Float-Zahlen während der Transformation. Die voreingestellte Genauigkeit entspricht `precision_fractional = 3` und kann jederzeit angepasst werden. 

# Herzlichen Glückwunsch!!! - Zeit, der Community beizutreten! 

Herzlichen Glückwunsch zum Abschluss dieses Notebook-Tutorials! Wenn es Ihnen gefallen hat und Sie sich der Bewegung zur Wahrung der Privatsphäre, zum dezentralisiertenen Besitz von KI und der KI-Lieferkette (Daten) anschließen möchten, können Sie dies auf folgende Weise tun! 

### PySyft auf GitHub einen Stern geben! 

Der einfachste Weg, unserer Community zu helfen, besteht darin, die GitHub-Repos mit Sternen auszuzeichnen! Dies hilft, das Bewusstsein für die coolen Tools zu schärfen, die wir bauen. 

- [Gib PySyft einen Stern](https://github.com/OpenMined/PySyft)

### Mach mit bei Slack! 

Der beste Weg, um über die neuesten Entwicklungen auf dem Laufenden zu bleiben, ist, sich unserer Community anzuschließen! Sie können dies tun, indem Sie das Formular unter [http://slack.openmined.org](http://slack.openmined.org) ausfüllen.

### Treten Sie einem Code-Projekt bei! 

Der beste Weg, um zu unserer Community beizutragen, besteht darin, Entwickler zu werden! Sie können jederzeit zur PySyft GitHub Issues-Seite gehen und nach "Projects" filtern. Dies zeigt Ihnen alle Top-Level-Tickets und gibt einen Überblick darüber, an welchen Projekten Sie teilnehmen können! Wenn Sie nicht an einem Projekt teilnehmen möchten, aber ein wenig programmieren möchten, können Sie auch nach weiteren "einmaligen" Miniprojekten suchen, indem Sie nach GitHub-Problemen suchen, die als "good first issue" gekennzeichnet sind. 

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### Spenden

Wenn Sie keine Zeit haben, zu unserer Codebase beizutragen, aber dennoch Unterstützung leisten möchten, können Sie auch Unterstützer unseres Open Collective werden. Alle Spenden fließen in unser Webhosting und andere Community-Ausgaben wie Hackathons und Meetups! 

 - [OpenMined's Open Collective Page](https://opencollective.com/openmined)